# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [9]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [23]:
Data = pd.read_csv("../WeatherData.csv")
#Data.head()
newWeather_df = Data.rename(columns={"Max Temp": "Temp",
                                    "Winde Speed": "WindSpeed"})
newWeather_df.head()

,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,WindSpeed,Country,Date
0,0,Maragogi,-9.01,-35.22,77.65,78,0,18.28,BR,1603750586
1,1,Guerrero Negro,27.98,-114.06,74.93,37,2,17.92,MX,1603750927
2,2,Atuona,-9.80,-139.03,79.12,72,90,20.38,PF,1603750585
3,3,Omsukchan,62.53,155.80,15.51,94,100,7.45,RU,1603750928
4,4,Matara,5.95,80.54,77.20,90,73,7.36,LK,1603750928


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
lat_and_lon = Data[["Lat", "Lng"]].astype(float)
humidity = Data["Humidity"]
gmaps.configure(api_key = g_key)

In [13]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(lat_and_lon, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
perfectTemp = (newWeather_df.Temp < 80) & (newWeather_df.Temp > 70)
perfectCloudiness = newWeather_df.Cloudiness == 0
perfectWindSpeed = newWeather_df.WindSpeed < 10
perfectWeather = perfectTemp & perfectCloudiness & perfectWindSpeed


idealWeather = newWeather_df[perfectWeather]
idealWeather = idealWeather.dropna()
idealWeather = idealWeather.reset_index()
idealWeather.head(10)

,index,Unnamed: 0,City,Lat,Lng,Temp,Humidity,Cloudiness,WindSpeed,Country,Date
0,130,130,Makkah al Mukarramah,21.43,39.83,76.26,57,0,1.23,SA,1603750622
1,133,133,Inhambane,-23.86,35.38,72.79,89,0,5.86,MZ,1603750898
2,241,241,Chabahar,25.29,60.64,73.22,45,0,2.17,IR,1603750962
3,256,256,General Roca,-39.03,-67.58,75.20,25,0,4.70,AR,1603750927
4,275,275,Todos Santos,23.45,-110.22,79.00,67,0,8.01,MX,1603750966
5,293,293,Mandalī,33.75,45.55,71.29,21,0,7.83,IQ,1603750969
6,369,369,Belaguntha,19.88,84.65,70.70,68,0,2.62,IN,1603750980
7,383,383,Giza,30.01,31.21,75.20,73,0,5.82,EG,1603750982
8,397,397,Doha,25.29,51.53,79.00,74,0,4.70,QA,1603750591
9,446,446,Sakakah,29.97,40.21,73.40,10,0,6.93,SA,1603750991


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = idealWeather
hotel_df["Hotel Name"] = ""

In [32]:
params = {
 "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)

    name_address = name_address.json()

    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        hotel_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df.info()

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 12 columns):
index         19 non-null int64
Unnamed: 0    19 non-null int64
City          19 non-null object
Lat           19 non-null float64
Lng           19 non-null float64
Temp          19 non-null float64
Humidity      19 non-null int64
Cloudiness    19 non

In [33]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [36]:
# Add marker layer ontop of heat map

fig = gmaps.figure()

hotel_layer = gmaps.marker_layer(
    locations,info_box_content=[info_box_template.format(**row) for index, row in hotel_df.iterrows()]
)

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))